In [17]:
from osgeo import gdal
from osgeo import ogr# 矢量处理数据包(shp,json等)
from osgeo import osr# 投影坐标包
from osgeo import gdal_array
from osgeo import gdalconst

import json# json是存储和交换文本信息的语法，类似 XML
import requests# 网络数据获取(开源数据，不开源就用爬虫)
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data1="/content/drive/MyDrive/AI data/DEM/ShiChuan.tiff"
data2='/content/drive/MyDrive/AI data/DEM/srtm_25_02.img'

**Copy**

In [4]:
# open the original file
ds = gdal.Open(data2)

# definete a new fole
data_copy = "data_copy.tif"

# get a gdal driver
driver_tiff = gdal.GetDriverByName("GTiff")

# create a copy of the original raster
ds_copy = driver_tiff.CreateCopy(data_copy, ds, strict=0)

# basic raster info
#  查看size
print(ds_copy.RasterXSize,ds_copy.RasterYSize)
#  获得投影信息
print(ds_copy.GetProjection())
#  地理仿射变换参数
print(ds_copy.GetGeoTransform())

# # add new raster to qgis
# iface.addRasterLayer(ds_copy)

ds_copy = None

6000 6000
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
(-60.0, 0.0008333333333333334, 0.0, 55.0, 0.0, -0.0008333333333333334)


**Change Raster Value**

In [3]:
fn = "test_raster.tif"
fn_template = '/content/drive/MyDrive/AI data/DEM/srtm_25_02.tif'

# open template raster
ds_temp = gdal.Open(fn_template)

# get driver
driver_tiff = gdal.GetDriverByName('GTiff')

# cretate dataset
ds = driver_tiff.Create(fn,xsize=10,ysize=10,bands=1,eType=gdal.GDT_Float32)
ds.SetGeoTransform(ds_temp.GetGeoTransform())
ds.SetProjection(ds_temp.GetProjection())

# read raster band as a array
band1 = ds.GetRasterBand(1).ReadAsArray()
# change band value
band1[0,0] = 100.00
band1[1] = 20.00
band1[5] = 50.00
ds.GetRasterBand(1).WriteArray(band1)

ds = None

**Raster No data values**

像元值可正可负，可以是整型也可以是浮点型。像元中还可以使用 NoData 值来表示数据缺失。有时，栅格数据集中会包含一些您不想显示的均匀区域。这些区域包括边框、背景或其他被认为没有有效值的数据。尽管有时这些数据可能具有实数值，但其他时候会将其表示为 NoData 值。

In [ ]:
fn = '/content/drive/MyDrive/AI data/DEM/srtm_25_02.tif'
newfn = 'no_data.tif'

# driver = gdal.GetDriverByName('GTiff')
# copyds = gdal.Open(fn)

# opy raster
# ds = driver_tiff.CreateCopy(newfn,copyds,strict=0)

# open new raster
#--------(can't change metadata, 只读，默认值为0)
# ds = gdal.Open(newfn)
#--------(could change metadata)
ds = gdal.Open(newfn,1)

# get nodata
ndv = ds.GetRasterBand(1).GetNoDataValue()

# set new nodata
newndv = -9999.0

# visual band
band1 = ds.GetRasterBand(1).ReadAsArray()
print(band1)

# change nodata
band1[band1 == ndv] = newndv

# visual new band
print(band1)

# change nodata
ds.GetRasterBand(1).SetNoDataValue(newndv)
# save new band
ds.GetRasterBand(1).WriteArray(band1)

**subtract raster layer**

将同一区域的不同年代的DEM相减

In [20]:


def openRaster(fn,access=0):
  ds = gdal.Open(fn,access)
  if ds is None:
    print('error opening raster dataset')
  return ds

def getRasterBand(fn,band=1,access=0):
  ds = openRaster(fn,access)
  band = ds.GetRasterBand(1).ReadAsArray()
  return band

def createRasterFromcopy(fn,ds,data,driverFmt='GTiff'):
  driver = gdal.GetDriverByName(driverFmt)
  outds = driver.CreateCopy(fn,ds,strict=0)
  outds.GetRasterBand(1).WriteArray(data)
  ds = None
  outds = None

def subtractRasters(fn1,fn2,fnout):
  data1 = getRasterBand(fn1)
  data2 = getRasterBand(fn2)
  dataout = data2-data1
  createRasterFromcopy(fnout,gdal.Open(fn1),dataout)


fn1 = '/content/drive/MyDrive/AI data/DEM/青藏高原/gebco_2019_n31.0_s30.5_w90.0_e90.5.tif'
fn2 = '/content/drive/MyDrive/AI data/DEM/青藏高原/gebco_2023_n31.0_s30.5_w90.0_e90.5.tif'
fnout = 'subtract.tif'
subtractRasters(fn1,fn2,fnout)

# 显示在QGIS
# iface.addRasterLayer(fnout)